In [ ]:
import re
import requests
from itertools import groupby

import pandas as pd
from geopy.geocoders import ArcGIS
from arcgis.gis import GIS
from arcgis.geocoding import geocode

api_key = "AAPK4d74de30452c4083822bac73c98a3c47pQecCTVCLZrikOxylcE0J26EvgdbFXBNOU-jPMjLdCF_GvHua3Aok7y0pmNoPdBf"
portal = GIS("https://www.arcgis.com", api_key=api_key)

raw_data = pd.read_csv("met_data_original.csv", dtype = str)
raw_data

pd.set_option('display.max_columns', None)

In [ ]:
# checked all possible geotypes --> will create new dataframe with origins + coordinates of each location
geo_df = raw_data[['Object Number', 'Object ID', 'Title', 'Geography Type', 'City',	'State', 'County', 'Country', 'Region', 'Subregion']].copy()

for i in geo_df.index:    

    extract_loc = [geo_df["City"].iloc[i], 
                    geo_df["Subregion"].iloc[i], 
                    geo_df["State"].iloc[i], 
                    geo_df["County"].iloc[i]]

    if type(geo_df["Country"].iloc[i]) != float:
        extract_loc.append(geo_df["Country"].iloc[i])
    elif type(geo_df["Region"].iloc[i]) != float:
        extract_loc.append(geo_df["Region"].iloc[i])

    base_loc = [p for p in extract_loc if not(pd.isnull(p)) == True]
    base_loc = [s.replace('Mesopotamia', 'Iraq') for s in base_loc]
    base_loc = [s.replace('Northern Syria', 'Syria') for s in base_loc]

    rm_lines = ['|', '||']
    base_loc =  [line for line in base_loc if line not in rm_lines]

    joined_loc = []
    for l in base_loc:
        if "|" in l: 
            if len(base_loc) == 1:
                split_loc = l.split("|")
                joined_loc = [key for key, _ in groupby(split_loc)]
            else:
                line_count = sum(s.count('|') for s in base_loc)
                if line_count >= len(base_loc):
                    l1 = []
                    l2 = []
                    for l in base_loc:
                        catch = l.split('|')
                        if len(catch) > 1:
                            l1.append(catch[0])
                            l2.append(catch[1])
                    join1 = ', '.join(l1)
                    join2 = ', '.join(l2)
                    joined_loc = [join1, join2]
        else:
            if " or " in l or " and " in l and "kingdom" not in l:
                print(base_loc)
                index = base_loc.index(l)
                ctxt = ', '.join(base_loc[index + 1:])
                base_loc.remove(l)
                if index == 0:
                    split_loc = []
                    if " or " in l:
                        split_loc = l.split(" or ")
                    
                    if " and " in l:
                        split_loc = l.split(" and ")
            
                    insert_pos = 0
                    for w in split_loc:
                        if len(ctxt) > 0:
                            joined_loc.insert(index + insert_pos, w + ", " + ctxt)
                        else:
                            joined_loc.insert(index + insert_pos, w)
                        insert_pos += 1
            else:
                joined_loc.append(', '.join(base_loc))

        for j in joined_loc:
            print(j)
            geo_df.loc[i, "origin_" + str(joined_loc.index(j))] = j


In [ ]:
# geocode each origin
for i, row_loc in enumerate(geo_df['origin_0']):
    if type(row_loc) != float:
        try:
            coords = geocode(address=row_loc)[0]['location']
            geo_df.loc[i, "lat_0"] = coords['y']
            geo_df.loc[i, "long_0"] = coords['x']
            print(str(coords['y']) + ', ' + str(coords['x']))
        except:
            geo_df.loc[i, "lat_0"] = "Unspecified"
            geo_df.loc[i, "long_0"] = "Unspecified"


for i, row_loc in enumerate(geo_df['origin_1']):
    if type(row_loc) != float:
        try:
            coords = geocode(address=row_loc)[0]['location']
            geo_df.loc[i, "lat_1"] = coords['y']
            geo_df.loc[i, "long_1"] = coords['x']
            print(str(coords['y']) + ', ' + str(coords['x']))
        except:
            geo_df.loc[i, "lat_1"] = "Unspecified"
            geo_df.loc[i, "long_1"] = "Unspecified"


for i, row_loc in enumerate(geo_df['origin_2']):
    if type(row_loc) != float:
        try:
            coords = geocode(address=row_loc)[0]['location']
            geo_df.loc[i, "lat_2"] = coords['y']
            geo_df.loc[i, "long_2"] = coords['x']
            print(str(coords['y']) + ', ' + str(coords['x']))
        except:
            geo_df.loc[i, "lat_2"] = "Unspecified"
            geo_df.loc[i, "long_2"] = "Unspecified"


for i, loc in enumerate(geo_df['origin_3']):
    if type(row_loc) != float:
        try:
            coords = geocode(address=row_loc)[0]['location']
            geo_df.loc[i, "lat_3"] = coords['y']
            geo_df.loc[i, "long_3"] = coords['x']
            print(str(coords['y']) + ', ' + str(coords['x']))
        except:
            geo_df.loc[i, "lat_3"] = "Unspecified"
            geo_df.loc[i, "long_3"] = "Unspecified"

In [ ]:
# save for safety
geo_df.to_csv('geodf_save.csv')

In [ ]:
# rejoin with original info
geo_df.drop(['Object Number', 'Title', 'Geography Type', 'City', 'State', 'County', 'Country', 'Region', 'Subregion'], axis=1, inplace=True)
joined_data = raw_data.merge(geo_df, on='Object ID')
joined_data

In [ ]:
# rm unnecessary columns
joined_data.drop(['Is Highlight', 'Is Timeline Work', 'Is Public Domain', 'Gallery Number', 'Dynasty', 'Reign', 'Portfolio', 'Constituent ID',	'Artist Prefix', 'Artist Suffix', 'Artist Alpha Sort', 'Artist Begin Date',	'Artist End Date',	'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL', 'Object Begin Date', 'Object End Date', 'Rights and Reproduction', 'Object Wikidata URL', 'Repository', 'Tags', 'Tags AAT URL', 'Tags Wikidata URL'], axis=1, inplace=True)
joined_data = joined_data.dropna(axis=0, subset=['origin_0'])

coords = geocode(address="Metropolitan Museum of Art, New York, NY")[0]['location']
joined_data['present_loc_lat'] = coords['y']
joined_data['present_loc_long'] = coords['x']

joined_data 

In [ ]:
# add link to images if available 
for i, row in enumerate(joined_data['Object ID']):
    response = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects/' + joined_data.loc[i, 'Object ID'])
    get_info = response.json()
    if 'primaryImageSmall' in get_info:
        joined_data.loc[i, '<img>-tooltip'] = get_info['primaryImageSmall']
    print(i)

In [ ]:
# export
joined_data.to_csv('geocode_met_data.csv')